In [1]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
import copy 
import numpy as np

In [2]:
def ucitaj_povrsinske_proteine(putanja_do_fajla):
    povrsinski_proteini = {}
    with open(putanja_do_fajla, "r") as fajl:
        for red in SeqIO.parse(fajl, "fasta"):
            if any(kljucna_rec in red.description for kljucna_rec in ['spike', 'surface']):
                povrsinski_proteini[red.description] = red.seq

    return povrsinski_proteini

In [3]:
putanja_do_fajla = "podaci/proteini/rnk.fasta"
povrsinski_proteini = ucitaj_povrsinske_proteine(putanja_do_fajla)

In [4]:
def rnk_u_kodone(rnk):
    kodoni = [rnk[i:i+3] for i in range(0, len(rnk), 3)]
    return kodoni

S obzirom da duzine rnk koje kodiraju povrsinske proteine nisu jednake duzine, moramo da im izjednacimo duzine kako bismo koristili p-adicno ili Hamingovo rastojanje.

In [5]:
# 1. pristup: kracu sekvencu dopunimo do duzine duze sekvence najucestalijim kodonom
def izjednacavanje_sekvenci_kodonima(sekv1, sekv2):
    sekv1, sekv2 = rnk_u_kodone(sekv1), rnk_u_kodone(sekv2)
    
    duza_sekvenca = sekv1 if len(sekv1) > len(sekv2) else sekv2
    kraca_sekvenca = sekv1 if len(duza_sekvenca) == len(sekv2) else sekv2
    ucestalost_kodona = {}
    
    for kodon in kraca_sekvenca:
        if kodon in ucestalost_kodona:
            ucestalost_kodona[kodon] += 1
        else:
            ucestalost_kodona[kodon] = 1
        
    najcesci_kodon = max(ucestalost_kodona, key=ucestalost_kodona.get)
    broj_pojavljivanja = ucestalost_kodona[najcesci_kodon]

    broj_kodona_dodati = len(duza_sekvenca) - len(kraca_sekvenca)
    dopunjeni_kodoni = [najcesci_kodon] * broj_kodona_dodati

    # da se ne bi iz nekog razloga direktno menjale vrednosti, pravimo kopiju 
    dopunjena_sekvenca = kraca_sekvenca.copy()
    dopunjena_sekvenca.extend(dopunjeni_kodoni)

    dopunjena_sekvenca = ''.join(str(seq) for seq in dopunjena_sekvenca)
    duza_sekvenca = ''.join(str(seq) for seq in duza_sekvenca)
    
    return dopunjena_sekvenca, duza_sekvenca


# 2. pristup: kracu sekvencu dopunimo do duzine duze sekvence najucestalijim nukleotidom
def izjednacavanje_sekvenci_nukleotidima(sek1, sek2):
    duza_sekvenca = sek1 if len(sek1) > len(sek2) else sek2
    kraca_sekvenca = sek1 if len(duza_sekvenca) == len(sek2) else sek2
    ucestalost_nukleotida = {}
    
    for nukleotid in kraca_sekvenca:
        if nukleotid in ucestalost_nukleotida:
            ucestalost_nukleotida[nukleotid] += 1
        else:
            ucestalost_nukleotida[nukleotid] = 1
        
    najcesci_nukleotid = max(ucestalost_nukleotida, key=ucestalost_nukleotida.get)

    broj_nukleotida_dodati = len(duza_sekvenca) - len(kraca_sekvenca)
    dopunjeni_nukleotidi = [najcesci_nukleotid] * broj_nukleotida_dodati
    
    dopunjena_sekvenca = kraca_sekvenca + ''.join(dopunjeni_nukleotidi)
        
    return dopunjena_sekvenca, duza_sekvenca

### P-adično rastojanje

In [6]:
# Prevodjenje kodona u p-adicne brojeve
def kodon_u_p_adicni_broj(kodon):
    kodon_u_broj = {
        'CCC': 111, 'CCU': 113, 'CCA': 112, 'CCG': 114,
        'CAC': 121, 'CAU': 123, 'CAA': 122, 'CAG': 124,
        'CUC': 131, 'CUU': 133, 'CUA': 132, 'CUG': 134,
        'CGC': 141, 'CGU': 143, 'CGA': 142, 'CGG': 144,
        'ACC': 211, 'ACU': 213, 'ACA': 212, 'ACG': 214,
        'AAC': 221, 'AAU': 223, 'AAA': 222, 'AAG': 224,
        'AUC': 231, 'AUU': 233, 'AUA': 232, 'AUG': 234,
        'AGC': 241, 'AGU': 243, 'AGA': 242, 'AGG': 244,
        'UCC': 311, 'UCU': 313, 'UCA': 312, 'UCG': 314,
        'UAC': 321, 'UAU': 323, 'UAA': 322, 'UAG': 324,
        'UUC': 331, 'UUU': 333, 'UUA': 332, 'UUG': 334,
        'UGC': 341, 'UGU': 343, 'UGA': 342, 'UGG': 344,
        'GCC': 411, 'GCU': 413, 'GCA': 412, 'GCG': 414,
        'GAC': 421, 'GAU': 423, 'GAA': 422, 'GAG': 424,
        'GUC': 431, 'GUU': 433, 'GUA': 432, 'GUG': 434,
        'GGC': 441, 'GGU': 443, 'GGA': 442, 'GGG': 444
    }

    return kodon_u_broj[kodon]
        
# Prevodjenje RNK u niz p-adicnih brojeva
def rnk_sekvenca_u_p_adicne_brojeve(rnk_sekvenca):
    brojevi = []
    for i in range(0, len(rnk_sekvenca), 3):
        kodon = rnk_sekvenca[i:i+3]
        brojevi.append(kodon_u_p_adicni_broj(kodon))  
    return brojevi

In [7]:
def p_adicno_rastojanje_kodona(x, y, p):
    x, y = str(x), str(y)
    
    if x[0] != y[0]:
        return 1
    elif x[1] != y[1]:
        return 1/p
    elif x[2] != y[2]:
        return 1/(p**2)

    return 0


def p_adicno_rastojanje(p_rnk1, p_rnk2, p=5):
    p_rastojanje = 0

    for (x, y) in zip(p_rnk1, p_rnk2):
        p_rastojanje += p_adicno_rastojanje_kodona(x, y, p)
           
    return p_rastojanje

In [8]:
# kod racunanja p-adicnog rastojanja, kracu sekvencu cemo, nakon prevodjenja u p-adicne brojeve, dopuniti 000.
# Kako p-adicna vrednost 000 ne postoji, to ce predstavljati dobar indikator dda su na tim pozicijama sekvence razlicite.

def izjednacavanje_sekvenci_nulama(sek1, sek2):
    duza_sekvenca = sek1 if len(sek1) > len(sek2) else sek2
    kraca_sekvenca = sek1 if len(duza_sekvenca) == len(sek2) else sek2
    
    broj_nula_dodati = len(duza_sekvenca) - len(kraca_sekvenca)
    dopunjene_nule = ['000'] * broj_nula_dodati
    
    dopunjena_sekvenca = kraca_sekvenca + dopunjene_nule
    
    return dopunjena_sekvenca, duza_sekvenca

In [9]:
# 1. pristup
print('p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_kodonima(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        p_rastojanje = p_adicno_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1.split(" | ")[1]} - {virus2.split(" | ")[1]} : {round(p_rastojanje, 5)}')


print()

        
 # 2. pristup
print('p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim nukleotidom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_nukleotidima(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        p_rastojanje = p_adicno_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1.split(" | ")[1]} i-{virus2.split(" | ")[1]} : {round(p_rastojanje, 5)}') 
        

print()


# 3. pristup
print('p-adicna rastojanja kada krace sekvence dopunjujemo nulama')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)
        
        rnk1_p_brojevi, rnk2_p_brojevi = izjednacavanje_sekvenci_nulama(rnk1_p_brojevi, rnk2_p_brojevi)
        
        p_rastojanje = p_adicno_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1.split(" | ")[1]} - {virus2.split(" | ")[1]} : {round(p_rastojanje, 5)}') 

p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom
Rastojanje izmedju:
	spike structural protein [Bovine coronavirus] - surface glycoprotein [Human coronavirus 229E] : 1029.32
	spike structural protein [Bovine coronavirus] - spike surface glycoprotein [Human coronavirus OC43] : 966.44
	spike structural protein [Bovine coronavirus] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1095.76
	spike structural protein [Bovine coronavirus] - spike glycoprotein [SARS coronavirus Tor2] : 1042.68
	surface glycoprotein [Human coronavirus 229E] - spike surface glycoprotein [Human coronavirus OC43] : 1031.2
	surface glycoprotein [Human coronavirus 229E] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1058.64
	surface glycoprotein [Human coronavirus 229E] - spike glycoprotein [SARS coronavirus Tor2] : 957.36
	spike surface glycoprotein [Human coronavirus OC43] - spike protein [Middle East respiratory syndrome-related coronavi

### Hamingovo rastojanje

In [10]:
def hamingovo_rastojanje(a, b): # a i b su sekvence kodona ili aminokiselina
    a = np.array(list(a))
    b = np.array(list(b))

    haming = 0
    for x, y in zip(a, b):
        if str(x) != str(y):
            haming += 1
    
    return haming

#### elementi su kodoni

In [11]:
# 1. pristup
print('Hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_kodonima(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        haming = hamingovo_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1.split(" | ")[1]} - {virus2.split(" | ")[1]} : {haming}')


print()

# 2. pristup
print('Hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim nukleotidom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_nukleotidima(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        haming = hamingovo_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1.split(" | ")[1]}- {virus2.split(" | ")[1]} : {haming}')

Hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom
Rastojanje izmedju:
	spike structural protein [Bovine coronavirus] - surface glycoprotein [Human coronavirus 229E] : 1317
	spike structural protein [Bovine coronavirus] - spike surface glycoprotein [Human coronavirus OC43] : 1213
	spike structural protein [Bovine coronavirus] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1334
	spike structural protein [Bovine coronavirus] - spike glycoprotein [SARS coronavirus Tor2] : 1319
	surface glycoprotein [Human coronavirus 229E] - spike surface glycoprotein [Human coronavirus OC43] : 1300
	surface glycoprotein [Human coronavirus 229E] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1322
	surface glycoprotein [Human coronavirus 229E] - spike glycoprotein [SARS coronavirus Tor2] : 1218
	spike surface glycoprotein [Human coronavirus OC43] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1316
	spik

#### elementi su aminokiseline

In [12]:
print('hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]
        
        rnk1, rnk2 = izjednacavanje_sekvenci_kodonima(rnk1, rnk2)
        
        rnk1 = Seq(rnk1).translate()
        rnk2 = Seq(rnk2).translate()
        
        haming = hamingovo_rastojanje(rnk1, rnk2)
        print(f'\t{virus1.split(" | ")[1]} - {virus2.split(" | ")[1]} : {haming}') 
        
print()
        
print('hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim nukleotidom')
print('Rastojanje izmedju:')
for i in range(len(povrsinski_proteini) - 1):
    for j in range(i+1, len(povrsinski_proteini)):
        virus1, rnk1 = list(povrsinski_proteini.items())[i]
        virus2, rnk2 = list(povrsinski_proteini.items())[j]
        
        rnk1, rnk2 = izjednacavanje_sekvenci_nukleotidima(rnk1, rnk2)

        rnk1 = Seq(rnk1).translate()
        rnk2 = Seq(rnk2).translate()
        
        
        haming = hamingovo_rastojanje(rnk1, rnk2)
        print(f'\t{virus1.split(" | ")[1]} - {virus2.split(" | ")[1]} : {haming}') 

hamingova rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom
Rastojanje izmedju:
	spike structural protein [Bovine coronavirus] - surface glycoprotein [Human coronavirus 229E] : 1269
	spike structural protein [Bovine coronavirus] - spike surface glycoprotein [Human coronavirus OC43] : 1163
	spike structural protein [Bovine coronavirus] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1283
	spike structural protein [Bovine coronavirus] - spike glycoprotein [SARS coronavirus Tor2] : 1267
	surface glycoprotein [Human coronavirus 229E] - spike surface glycoprotein [Human coronavirus OC43] : 1254
	surface glycoprotein [Human coronavirus 229E] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1282
	surface glycoprotein [Human coronavirus 229E] - spike glycoprotein [SARS coronavirus Tor2] : 1162
	spike surface glycoprotein [Human coronavirus OC43] - spike protein [Middle East respiratory syndrome-related coronavirus] : 1260
	spik

In [13]:
# Ocekivano je da je rastojanje gde poredimo aminokiseline manje nego rastojanje gde poredimo kodone zato sto
# jednu aminokiselinu kodira vise kodona.

# Nije velika razlika u rastojanju ako se kraca sekvenca dopuni kodonima ili nukleotidima (ili nulama za 
# racunanje p-adicnog rastojanja)


# Zasto je manje 5-adicno rastojanje od hamingovog?
# Zato sto 5-adicno rastojanje najvise vrednuje razliku na prvom nukleotidu kodona (1), razlika na drugom 
# nukleotidu je 5 puta manja u odonosu na prvi, dok je razlika na trecem nukleotidu 25 puta manja. Kada se to 
# sumira rastojanje ne moze biti veca nego hamingovo.